# Install Pre-requisites

In [1]:
!pip install transformers --quiet

In [2]:
!pip install datasets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install rouge-score --quiet

  Preparing metadata (setup.py) ... done


In [4]:
!pip install bleu --quiet

  Preparing metadata (setup.py) ... done


In [5]:
!pip install bert-score --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00


In [6]:
!git clone https://github.com/google-research/bleurt.git

Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116 (from 1)
Receiving objects: 100% (134/134), 31.28 MiB | 24.34 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [7]:
cd bleurt

/content/bleurt


In [8]:
!pip install .

Processing /content/bleurt
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=548bcb6f2d64301bf63227be76d58c318002720e0a80f7e5d9dafd37c74a4a8c
  Stored in directory: /tmp/pip-ephem-wheel-cache-516ayfor/wheels/92/4f/fb/afa555fa27aa9e2c7958df797a62cc4e74f0f459cec9c4fa7c
Successfully built BLEURT


In [ ]:
# !python -m unittest bleurt.score_test
# !python -m unittest bleurt.score_not_eager_test
# !python -m unittest bleurt.finetune_test
# !python -m unittest bleurt.score_files_test


In [9]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset
from rouge_score import rouge_scorer
from bleurt import score as bleurt_score
from bleurt.score import BleurtScorer
from google.colab import drive
from bert_score import score as bert_score
drive.mount('/content/drive')

Mounted at /content/drive


# Globals

In [10]:
# full, test dataset select data slicing of original json
# global variables
model_name = 'presidio-longformer_mask' # update to select the right path
task = 'mask'
size = 'full'
split = 'test'
# replace,redact, hash, mask
anonymization = 'redact'

In [11]:
path_data = '/content/drive/MyDrive/266 - Natural Language Processing/Final Project/'
path_models = '/content/drive/MyDrive/266 - Natural Language Processing/Final Project/Models'

In [13]:
ds_data = load_dataset('json', data_files= f'{path_data}data/tab/echr_{model_name}_{split}_{size}_{task}_{anonymization}_masked_spans_gold.json')

#ds_data = load_dataset('json', data_files=f'{path_data}/presidio_anonymized_texts_with_defaults.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
ds_data

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'original_text', 'anonymized_text'],
        num_rows: 98
    })
})

# Evaluation

In [19]:
# Load json
# Initialize BLEURT scorer with the local checkpoint

#bleurt_scorer = BleurtScorer(bleurt_checkpoint)

In [21]:
# Initialize the Pegasus model and tokenizer
model_name = 'google/pegasus-large'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Initialize BLEURT scorer with the local checkpoint
#bleurt_checkpoint = path_models + "/bleurt-base-128/" # Path to downloaded BLEURT checkpoint
bleurt_checkpoint = "/content/bleurt/bleurt/test_checkpoint" # Path to downloaded BLEURT checkpoint
bleurt_scorer = BleurtScorer(bleurt_checkpoint)

def generate_summary(text):
    """Generate a summary from the Pegasus model."""
    input_ids = tokenizer(text, return_tensors='pt', max_length=512, truncation=True).input_ids
    with torch.no_grad():
        summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def bert_similarity(original_text, summary):
    """Calculate BERTScore similarity between the original text and the summary."""
    P, R, F1 = bert_score([summary], [original_text], lang="en", verbose=True)
    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item()
    }

def bleurt_similarity(original_text, summary):
    """Calculate BLEURT score between the original text and the summary."""
    return bleurt_scorer.score(references=[original_text], candidates=[summary])[0]  # Returns a single score

def rouge_recall(original_text, summary):
    """Calculate ROUGE-Recall score between the original text and the summary."""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(original_text, summary)
    rouge_recall_scores = {
        'rouge1_recall': scores['rouge1'].recall,
        'rouge2_recall': scores['rouge2'].recall,
        'rougeL_recall': scores['rougeL'].recall
    }
    return rouge_recall_scores

# Initialize accumulators for average calculation
total_original_bert = {'precision': 0, 'recall': 0, 'f1': 0}
total_anonymized_bert = {'precision': 0, 'recall': 0, 'f1': 0}
total_original_bleurt = 0
total_anonymized_bleurt = 0
total_original_rouge_recall = {'rouge1_recall': 0, 'rouge2_recall': 0, 'rougeL_recall': 0}
total_anonymized_rouge_recall = {'rouge1_recall': 0, 'rouge2_recall': 0, 'rougeL_recall': 0}
num_entries = len(ds_data["train"])

# Process each entry in the test dataset
for entry in ds_data["train"]:
    original_text = entry['original_text']
    anonymized_text = entry['anonymized_text']

    # Generate summaries for original and anonymized texts
    #generated_summary_from_original = generate_summary(original_text)
    generated_summary_from_anonymized = generate_summary(anonymized_text)

    # Calculate evaluation metrics for the summary from the original text
    #original_bert = bert_similarity(original_text, generated_summary_from_original)
    #original_bleurt = bleurt_similarity(original_text, generated_summary_from_original)
    #original_rouge_recall = rouge_recall(original_text, generated_summary_from_original)

    # Accumulate scores for the original text
    #for key in total_original_bert:
    #    total_original_bert[key] += original_bert[key]
    #total_original_bleurt += original_bleurt
    #for key in total_original_rouge_recall:
    #    total_original_rouge_recall[key] += original_rouge_recall[key]

    # Calculate evaluation metrics for the summary from the anonymized text
    anonymized_bert = bert_similarity(original_text, generated_summary_from_anonymized)
    anonymized_bleurt = bleurt_similarity(original_text, generated_summary_from_anonymized)
    anonymized_rouge_recall = rouge_recall(original_text, generated_summary_from_anonymized)

    # Accumulate scores for the anonymized text
    for key in total_anonymized_bert:
        total_anonymized_bert[key] += anonymized_bert[key]
    total_anonymized_bleurt += anonymized_bleurt
    for key in total_anonymized_rouge_recall:
        total_anonymized_rouge_recall[key] += anonymized_rouge_recall[key]

# Calculate average scores
#average_original_bert = {key: total_original_bert[key] / num_entries for key in total_original_bert}
average_anonymized_bert = {key: total_anonymized_bert[key] / num_entries for key in total_anonymized_bert}
#average_original_bleurt = total_original_bleurt / num_entries
average_anonymized_bleurt = total_anonymized_bleurt / num_entries
#average_original_rouge_recall = {key: total_original_rouge_recall[key] / num_entries for key in total_original_rouge_recall}
average_anonymized_rouge_recall = {key: total_anonymized_rouge_recall[key] / num_entries for key in total_anonymized_rouge_recall}

# Display average scores
#print("Average Scores for Summaries from Original Text:")
#print("BERTScore:", average_original_bert)
#print("BLEURT Score:", average_original_bleurt)
#print("ROUGE-Recall:", average_original_rouge_recall)

print("\nAverage Scores for Summaries from Anonymized Text:")
print("BERTScore:", average_anonymized_bert)
print("BLEURT Score:", average_anonymized_bleurt)
print("ROUGE-Recall:", average_anonymized_rouge_recall)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.22 seconds, 4.60 sentences/sec
Average Scores for Summaries from Original Text:
BERTScore: {'precision': 0.92889517484879, 'recall': 0.8359980449384573, 'f1': 0.8799591392886882}
BLEURT Score: -0.7538957644482048
ROUGE-Recall: {'rouge1_recall': 0.12305995474547653, 'rouge2_recall': 0.1185378205198218, 'rougeL_recall': 0.12301255457661467}

Average Scores for Summaries from Anonymized Text:
BERTScore: {'precision': 0.8669517678873879, 'recall': 0.8126210162834245, 'f1': 0.8387986604048281}
BLEURT Score: -0.9753698195729937
ROUGE-Recall: {'rouge1_recall': 0.09969469778010638, 'rouge2_recall': 0.09087074429828626, 'rougeL_recall': 0.0996298593073829}
